# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load the csv exported in Part I to a DataFrame
vacation_df=pd.read_csv(output_data_file)
vacation_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity(%),Cloudiness,Wind Speed,Country,Date
0,0,morant bay,17.88,-76.41,82.99,88,42,1.01,JM,1603922850
1,1,luderitz,-26.65,15.16,60.60,67,90,13.47,NaN,1603922791
2,2,acapulco,16.86,-99.89,84.87,70,48,7.76,MX,1603922930
3,3,nanortalik,60.14,-45.24,32.85,92,100,14.83,GL,1603922970
4,4,west helena,34.55,-90.64,62.60,100,100,5.82,US,1603923010


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)
locations=vacation_df[["Latitude","Longitude"]]
weight=vacation_df["Humidity(%)"].astype(float)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weight,
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)
#Add Heatmap layer to map.
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
fit_weather_df=vacation_df.loc[(vacation_df["Temperature"]>70) &
                              (vacation_df["Temperature"]<80) &
                              (vacation_df["Wind Speed"]<10) &
                              (vacation_df["Cloudiness"]==0)].dropna()
fit_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity(%),Cloudiness,Wind Speed,Country,Date
48,48,porbandar,21.64,69.61,77.59,67,0,9.48,IN,1603924807
59,59,angoche,-16.23,39.91,75.45,79,0,1.57,MZ,1603925476
79,79,tezu,27.92,96.17,70.20,47,0,3.40,IN,1603926278
98,98,doha,25.29,51.53,77.00,59,0,1.99,QA,1603926803
100,100,minab,27.15,57.08,70.23,21,0,5.88,IR,1603927120
110,110,keti bandar,24.14,67.45,77.67,66,0,5.82,PK,1603927562
147,147,tura,25.52,90.22,70.05,90,0,4.00,IN,1603928828
211,211,bilma,18.69,12.92,70.14,20,0,4.50,NE,1603931595
233,233,asyut,27.18,31.18,71.49,64,0,9.13,EG,1603932779
253,253,birao,10.28,22.79,74.01,39,0,1.86,CF,1603933741


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=fit_weather_df[["City","Country","Latitude","Longitude"]].copy()
hotel_df["Hotel Name"]=""                   
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
48,porbandar,IN,21.64,69.61,
59,angoche,MZ,-16.23,39.91,
79,tezu,IN,27.92,96.17,
98,doha,QA,25.29,51.53,
100,minab,IR,27.15,57.08,


In [13]:
params = {"radius": 5000,
   "types": "lodging",
   "key": g_key}

for index, row in hotel_df.iterrows():
   lat = row["Latitude"]
   lng = row["Longitude"]
   params["location"] = f"{lat},{lng}"
    
   base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   response = requests.get(base_url, params=params)
   response = response.json()

   try:
       hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
   except (KeyError, IndexError):
       print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
48,porbandar,IN,21.64,69.61,Lords Inn Porbandar
59,angoche,MZ,-16.23,39.91,Guest House HESADA
79,tezu,IN,27.92,96.17,Circuit House
98,doha,QA,25.29,51.53,Doha Marriott Hotel
100,minab,IR,27.15,57.08,فروشگاه موبایل ترکمانی
110,keti bandar,PK,24.14,67.45,Haji laloo memon house
147,tura,IN,25.52,90.22,Hotel RIKMAN Continental
211,bilma,NE,18.69,12.92,Expeditions Ténére voyages
233,asyut,EG,27.18,31.18,Florence Hotel Assiut
253,birao,CF,10.28,22.79,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))